In [22]:
import yfinance as yf
import pandas as pd
from datetime import timedelta

In [23]:
def get_portfolio_performance(tickers, buy_date, sell_date, initial_investment):
    investment_per_stock = initial_investment / len(tickers)
    results = []

    print(f"Analyzing portfolio from {buy_date} to {sell_date}...")
    
    for ticker_symbol in tickers:
        stock = yf.Ticker(ticker_symbol)
        
        # Fetch history including dividends
        # We fetch a slightly wider range to ensure we capture the closest trading days
        start_dt = pd.to_datetime(buy_date)
        end_dt = pd.to_datetime(sell_date)
        
        hist = stock.history(start=start_dt, end=end_dt + timedelta(days=1))
        
        if hist.empty:
            print(f"Warning: No data found for {ticker_symbol}. Skipping.")
            continue

        # Get the closing price on the actual buy/sell dates (or the nearest trading day)
        buy_price = hist.iloc[0]['Close']
        sell_price = hist.iloc[-1]['Close']
        
        # Calculate shares (assuming fractional shares for exact equal weighting)
        shares_owned = investment_per_stock / buy_price
        
        # Calculate total dividends received during the holding period
        # yfinance 'Dividends' column shows the amount per share on the ex-dividend date
        total_dividends_per_share = hist['Dividends'].sum()
        total_dividends_received = total_dividends_per_share * shares_owned
        
        # P/L Calculation
        final_value = (shares_owned * sell_price) + total_dividends_received
        profit_loss = final_value - investment_per_stock
        percentage_return = (profit_loss / investment_per_stock) * 100
        
        results.append({
            'Ticker': ticker_symbol,
            'Buy Price': round(buy_price, 2),
            'Sell Price': round(sell_price, 2),
            'Dividends Rec.': round(total_dividends_received, 2),
            'P/L ($)': round(profit_loss, 2),
            'Return (%)': round(percentage_return, 2)
        })

    # Create Summary DataFrame
    df = pd.DataFrame(results)
    total_pl = df['P/L ($)'].sum()
    total_return = (total_pl / initial_investment) * 100
    
    return df, total_pl, total_return

In [58]:
import pandas as pd
import yfinance as yf
from datetime import timedelta

def get_portfolio_performance(tickers, buy_date, sell_date, initial_investment):
    investment_per_stock = initial_investment / len(tickers)
    results = []

    print(f"Analyzing portfolio from {buy_date} to {sell_date}...")
    
    for ticker_symbol in tickers:
        stock = yf.Ticker(ticker_symbol)
        
        start_dt = pd.to_datetime(buy_date)
        end_dt = pd.to_datetime(sell_date)
        
        # CHANGE 1: Set auto_adjust=False to explicitly get the 'Adj Close' column
        hist = stock.history(start=start_dt, end=end_dt + timedelta(days=1), auto_adjust=False)
        
        if hist.empty:
            print(f"Warning: No data found for {ticker_symbol}. Skipping.")
            continue

        # CHANGE 2: Use 'Adj Close' (Total Return Price) instead of 'Close'
        # We use a quick check to fallback to 'Close' if 'Adj Close' is missing (rare)
        price_col = 'Adj Close' if 'Adj Close' in hist.columns else 'Close'
        
        buy_price = hist.iloc[0][price_col]
        sell_price = hist.iloc[-1][price_col]
        
        shares_owned = investment_per_stock / buy_price
        
        # REMOVED: Dividend fetching and calculation logic
        
        # CHANGE 3: P/L is now derived purely from the change in Adjusted Price
        final_value = (shares_owned * sell_price) 
        profit_loss = final_value - investment_per_stock
        percentage_return = (profit_loss / investment_per_stock) * 100
        
        results.append({
            'Ticker': ticker_symbol,
            'Buy Price': round(buy_price, 2),
            'Sell Price': round(sell_price, 2),
            'P/L ($)': round(profit_loss, 2),
            'Return (%)': round(percentage_return, 2)
        })

    # Create Summary DataFrame
    df = pd.DataFrame(results)
    
    # Handle case where all stocks were skipped to prevent crash
    if df.empty:
        return df, 0.0, 0.0

    total_pl = df['P/L ($)'].sum()
    total_return = (total_pl / initial_investment) * 100
    
    return df, total_pl, total_return

In [51]:
tickers_by_year = {
    2017: ["CGNX", "ISRG", "EXAS", "MRVL", "BA", "PANW", "GRPN", "HQY", "POWI", "SRPT"],
    2018: ["FTNT", "GH", "INGN", "GKOS", "PAYC", "ROL", "ISRG", "TNDM", "HQY", "PRLB"],
    2019: ["RMD", "NEWR", "PANW", "HUBS", "CDW", "DOCU", "ZTS", "PAYC", "PRO", "SSD"],
    2020: ["PAYC", "DDOG", "UPWK", "HUBS", "TTC", "PD", "AAPL", "MRNA", "BAND", "WDAY"],
    2021: ["MTD", "PCTY", "FOXF", "WAT", "TTD", "STX", "PGNY", "HRI", "PAYC", "SQ"],
    2022: ["UNH", "UNFI", "WM", "ENSG", "MUSA", "URI", "CVS", "GMS", "CNC", "BLDR"],
    2023: ["DY", "MUSA", "RS", "KLAC", "EME", "WCC", "FIX", "DE", "NSIT", "URI"],
    2024: ['DY', 'FIX', 'EME', 'GMS', 'BLDR', 'IESC', 'PWR', 'CNM', 'KLAC', 'BLD']
}

In [53]:
tickers_by_year = {
    2024: ['DY', 'EME', 'FIX', 'RS', 'JBL', 'MUSA', 'SFM', 'NSIT', 'LBRT', 'BMI']
}

In [59]:
tickers_by_year = {
    2017: ['CCMP', 'SSD', 'HP', 'UNF', 'SHO', 'MRVL', 'SLAB', 'TER', 'FNSR', 'GMED'],
    2018: ['MRVL', 'FFIV', 'HQY', 'BA', 'PANW', 'NEOG', 'PAYC', 'MON_delisted', 'INGN', 'SWKS'],
    2019: ['ROL', 'BA', 'GH', 'PAYC', 'GKOS', 'ISRG', 'PRLB', 'TNDM', 'BLKB', 'FFIV'],
    2020: ['PAYC', 'CDW', 'PANW', 'FIVN', 'HUBS', 'ZTS', 'DOCU', 'MDB', 'ZEN', 'AAPL'],
    2021: ['PCTY', 'TTD', 'PAYC', 'APPS', 'MGNI', 'TYL', 'NET', 'ESTC', 'AVLR', 'MANH'],
    2022: ['AAPL', 'MTD', 'PCTY', 'STX', 'SNPS', 'THC', 'WAT', 'NOW', 'COST', 'VRTV'],
    2023: ['UNH', 'WM', 'URI', 'FIX', 'FANG', 'DLTR', 'MUSA', 'CVS', 'DE', 'BG'],
    2024: ['FIX', 'DY', 'EME', 'SFM', 'MUSA', 'RS', 'JBL', 'NSIT', 'GMS', 'CBZ'],
}

In [60]:
for year in sorted(tickers_by_year):
    tickers = tickers_by_year[year]
    buy_dt = f"{year}-03-31"
    sell_dt = f"{year+1}-03-31"
    total_cash = 10000
    # Execute
    performance_df, total_dollar_pl, total_pct = get_portfolio_performance(tickers, buy_dt, sell_dt, total_cash)
    
    # Output Results
    print("\n--- Individual Stock Performance ---")
    print(performance_df.to_string(index=False))
    print("-" * 40)
    print(f"Total Portfolio P/L: ${total_dollar_pl:,.2f}")
    print(f"Total Portfolio Return: {total_pct:.2f}%")


Analyzing portfolio from 2017-03-31 to 2018-03-31...


$CCMP: possibly delisted; no timezone found


$FNSR: possibly delisted; no timezone found



--- Individual Stock Performance ---
Ticker  Buy Price  Sell Price  P/L ($)  Return (%)
   SSD      39.13       53.19   359.43       35.94
    HP      43.20       45.42    51.37        5.14
   UNF     135.18      154.64   143.96       14.40
   SHO      11.81       12.26    37.55        3.76
  MRVL      14.39       20.04   393.29       39.33
  SLAB      73.55       89.90   222.30       22.23
   TER      29.66       43.93   481.02       48.10
  GMED      29.62       49.82   681.97       68.20
----------------------------------------
Total Portfolio P/L: $2,370.89
Total Portfolio Return: 23.71%
Analyzing portfolio from 2018-03-31 to 2019-03-31...


$MON_DELISTED: possibly delisted; no timezone found


...


--- Individual Stock Performance ---
Ticker  Buy Price  Sell Price  P/L ($)  Return (%)
  PCTY     179.83      205.77   144.25       14.42
   TTD      65.17       69.25    62.67        6.27
  PAYC     362.71      339.50   -63.99       -6.40
  APPS      80.36       43.81  -454.83      -45.48
  MGNI      41.61       13.21  -682.53      -68.25
   TYL     424.53      444.89    47.96        4.80
   NET      70.26      119.70   703.67       70.37
  ESTC     111.20       88.95  -200.09      -20.01
  MANH     117.38      138.71   181.72       18.17
----------------------------------------
Total Portfolio P/L: $-261.17
Total Portfolio Return: -2.61%
Analyzing portfolio from 2022-03-31 to 2023-03-31...


$VRTV: possibly delisted; no timezone found



--- Individual Stock Performance ---
Ticker  Buy Price  Sell Price  P/L ($)  Return (%)
  AAPL     171.24      162.70   -49.89       -4.99
   MTD    1373.19     1530.21   114.35       11.43
  PCTY     205.77      198.78   -33.97       -3.40
   STX      79.38       61.15  -229.63      -22.96
  SNPS     333.27      386.25   158.97       15.90
   THC      85.96       59.42  -308.75      -30.87
   WAT     310.39      309.63    -2.45       -0.24
   NOW     111.38       92.94  -165.51      -16.55
  COST     549.84      477.71  -131.19      -13.12
----------------------------------------
Total Portfolio P/L: $-648.07
Total Portfolio Return: -6.48%
Analyzing portfolio from 2023-03-31 to 2024-03-31...

--- Individual Stock Performance ---
Ticker  Buy Price  Sell Price  P/L ($)  Return (%)
   UNH     448.98      477.24    62.95        6.29
    WM     156.46      207.79   328.07       32.81
   URI     384.55      710.02   846.37       84.64
   FIX     144.40      315.76  1186.78      118.68
  FA

$GMS: possibly delisted; no timezone found



--- Individual Stock Performance ---
Ticker  Buy Price  Sell Price  P/L ($)  Return (%)
   FIX     317.53      321.59    12.77        1.28
    DY     139.63      152.34    91.03        9.10
   EME     353.49      369.09    44.12        4.41
   SFM      63.61      152.64  1399.62      139.96
  MUSA     423.48      467.82   104.71       10.47
    RS     323.95      285.10  -119.92      -11.99
   JBL     135.10      135.90     5.95        0.59
  NSIT     186.00      149.99  -193.60      -19.36
   CBZ      77.86       75.86   -25.69       -2.57
----------------------------------------
Total Portfolio P/L: $1,318.99
Total Portfolio Return: 13.19%


In [61]:
tickers = {
    '2024-01-31': ['EME', 'KLAC', 'FIX', 'DY', 'MUSA', 'SFM', 'RS', 'NSIT', 'PWR', 'LRCX'],
    '2024-02-29': ['DY', 'NSIT', 'SFM', 'MUSA', 'EME', 'FIX', 'CBZ', 'LRCX', 'KLAC', 'GMS'],
    '2024-03-31': ['FIX', 'DY', 'EME', 'SFM', 'MUSA', 'RS', 'JBL', 'NSIT', 'GMS', 'CBZ'],
    '2024-04-30': ['DY', 'HUBS', 'EME', 'RS', 'FIX', 'CBZ', 'FAST', 'NSIT', 'MUSA', 'SFM'],
    '2024-05-31': ['DY', 'KLAC', 'EME', 'NSIT', 'WCC', 'SFM', 'MUSA', 'LRCX', 'CMC', 'FIX'],
    '2024-06-30': ['DY', 'NSIT', 'KLAC', 'RS', 'EME', 'GMS', 'SFM', 'LBRT', 'PR', 'FAST'],
    '2024-07-31': ['DY', 'FIX', 'MUSA', 'KLAC', 'EME', 'PWR', 'RS', 'IESC', 'LBRT', 'NSIT'],
    '2024-08-31': ['DY', 'EME', 'FIX', 'RS', 'MUSA', 'KLAC', 'NSIT', 'HUBS', 'ZS', 'LRCX'],
    '2024-09-30': ['DY', 'EME', 'FIX', 'IESC', 'PWR', 'NTAP', 'KLAC', 'GMS', 'MANH', 'CBZ'],
    '2024-10-31': ['DY', 'FIX', 'EME', 'KLAC', 'IESC', 'NSIT', 'CNM', 'NTAP', 'PWR', 'BCO'],
    '2024-11-30': ['EME', 'DY', 'FIX', 'MUSA', 'PWR', 'KLAC', 'NSIT', 'BCO', 'CNM', 'HURN'],
    '2024-12-31': ['FIX', 'DY', 'EME', 'GMS', 'PWR', 'BLDR', 'IESC', 'KLAC', 'BLD', 'MUSA'],
}

In [68]:
tickers = {
    '2023-01-31': ['UNH', 'WM', 'BLDR', 'UNFI', 'MUSA', 'ASGN', 'GMS', 'CVS', 'CBZ', 'ENTG'],
    '2023-02-28': ['UNH', 'WM', 'DE', 'WMT', 'URI', 'UNFI', 'MUSA', 'ENSG', 'FANG', 'RSG'],
    '2023-03-31': ['UNH', 'WM', 'URI', 'FIX', 'FANG', 'DLTR', 'MUSA', 'CVS', 'DE', 'BG'],
    '2023-04-30': ['UNH', 'WM', 'MUSA', 'FIX', 'TGT', 'KR', 'DE', 'NSIT', 'CVS', 'UNFI'],
    '2023-05-31': ['UNH', 'KLAC', 'FIX', 'WM', 'KR', 'MUSA', 'MTD', 'EME', 'CVS', 'BLDR'],
    '2023-06-30': ['KLAC', 'FIX', 'TGT', 'XGN', 'UNH', 'DY', 'WM', 'URI', 'MUSA', 'WMT'],
    '2023-07-31': ['FIX', 'MUSA', 'KLAC', 'DY', 'TGT', 'EME', 'WM', 'URI', 'NSIT', 'VMI'],
    '2023-08-31': ['KLAC', 'FIX', 'DY', 'TGT', 'XGN', 'URI', 'UFPI', 'WM', 'MUSA', 'JBL'],
    '2023-09-30': ['DY', 'KLAC', 'FIX', 'EME', 'TRTN', 'NSIT', 'URI', 'CBZ', 'DE', 'MMM'],
    '2023-10-31': ['KLAC', 'FIX', 'DY', 'GMS', 'EME', 'VMI', 'URI', 'IRM', 'WM', 'NSIT'],
    '2023-11-30': ['EME', 'GMS', 'KLAC', 'FIX', 'DY', 'NSIT', 'SFM', 'VMI', 'URI', 'IRM'],
    '2023-12-31': ['DY', 'EME', 'MUSA', 'FIX', 'KLAC', 'DE', 'RS', 'PWR', 'WCC', 'LBRT'],
}

In [72]:
tickers = {
    '2022-01-31': ['PCTY', 'MTD', 'PAYC', 'GNRC', 'WAT', 'TTD', 'NTAP', 'FOXF', 'STX', 'SNPS'],
    '2022-02-28': ['MTD', 'PCTY', 'STX', 'GNRC', 'PAYC', 'WAT', 'SNPS', 'ENSG', 'TTD', 'THC'],
    '2022-03-31': ['AAPL', 'MTD', 'PCTY', 'STX', 'SNPS', 'THC', 'WAT', 'NOW', 'COST', 'VRTV'],
    '2022-04-30': ['AAPL', 'ESTA', 'MTD', 'SPR', 'COST', 'CRS', 'PCTY', 'STX', 'IDXX', 'NSIT'],
    '2022-05-31': ['AAPL', 'MTD', 'STX', 'PCTY', 'HRI', 'COST', 'WM', 'THC', 'SHC', 'MOH'],
    '2022-06-30': ['MTD', 'AAPL', 'CCMP', 'WAT', 'WM', 'STX', 'THC', 'HRI', 'IDXX', 'CRL'],
    '2022-07-31': ['AAPL', 'MTD', 'STX', 'HRI', 'CRL', 'WAT', 'THC', 'CCMP', 'URI', 'WM'],
    '2022-08-31': ['MTD', 'STX', 'URI', 'WM', 'THC', 'CCMP', 'HRI', 'IDXX', 'UNH', 'CSX'],
    '2022-09-30': ['WM', 'MTD', 'RSG', 'CSX', 'CCMP', 'URI', 'WAT', 'CNC', 'UNH', 'CBZ'],
    '2022-10-31': ['UNH', 'WM', 'MTD', 'CNC', 'URI', 'CSX', 'RSG', 'THC', 'MPLX', 'VRTV'],
    '2022-11-30': ['SUNW', 'UNH', 'WM', 'CVS', 'CNC', 'URI', 'MPLX', 'RSG', 'UNFI', 'AAPL'],
    '2022-12-31': ['UNH', 'WM', 'UNFI', 'URI', 'MUSA', 'CVS', 'ASGN', 'DLTR', 'ENSG', 'BLDR'],
}

In [75]:
tickers = {
    '2021-01-31': ['PAYC', 'UPWK', 'VERX', 'HUBS', 'DDOG', 'AAPL', 'NEWR', 'ENTG', 'WDAY', 'PD'],
    '2021-02-28': ['TTD', 'ZTS', 'PCTY', 'MTD', 'PAYC', 'CDNS', 'LLY', 'APPS', 'TYL', 'IDXX'],
    '2021-03-31': ['PCTY', 'TTD', 'PAYC', 'APPS', 'MGNI', 'TYL', 'NET', 'ESTC', 'AVLR', 'MANH'],
    '2021-04-30': ['PCTY', 'TTD', 'PAYC', 'APPS', 'AMD', 'MGNI', 'TYL', 'SQ', 'MANH', 'CDNS'],
    '2021-05-31': ['PCTY', 'TTD', 'PAYC', 'AMD', 'MTD', 'SQ', 'APPS', 'MANH', 'AAPL', 'CDNS'],
    '2021-06-30': ['MTD', 'PCTY', 'TTD', 'PAYC', 'AAPL', 'APPS', 'SQ', 'AMD', 'MANH', 'LRCX'],
    '2021-07-31': ['MTD', 'PCTY', 'AAPL', 'SQ', 'ENTG', 'PAYC', 'TTD', 'AMD', 'APPS', 'MANH'],
    '2021-08-31': ['MTD', 'PCTY', 'SNPS', 'PAYC', 'SQ', 'TTD', 'MANH', 'AMD', 'AAPL', 'APPS'],
    '2021-09-30': ['PCTY', 'MTD', 'TTD', 'PAYC', 'MANH', 'SQ', 'WAT', 'SPSC', 'SNPS', 'THC'],
    '2021-10-31': ['PCTY', 'MTD', 'PAYC', 'TTD', 'SNPS', 'WAT', 'SQ', 'SPSC', 'DVA', 'MANH'],
    '2021-11-30': ['PCTY', 'MTD', 'PAYC', 'TTD', 'STX', 'SQ', 'FOXF', 'GNRC', 'SNPS', 'MANH'],
    '2021-12-31': ['MTD', 'PCTY', 'FOXF', 'WAT', 'PAYC', 'STX', 'HRI', 'PGNY', 'SQ', 'SNPS'],
}

In [78]:
tickers = {
    '2020-01-31': ['PAYC', 'CDW', 'PANW', 'NEWR', 'HUBS', 'PRO', 'RMD', 'QTWO', 'FIVN', 'MDB'],
    '2020-02-29': ['CDW', 'PAYC', 'PANW', 'HUBS', 'PRO', 'EVBG', 'MDB', 'RMD', 'SQ', 'DOCU'],
    '2020-03-31': ['PAYC', 'CDW', 'PANW', 'FIVN', 'HUBS', 'ZTS', 'DOCU', 'MDB', 'ZEN', 'AAPL'],
    '2020-04-30': ['PAYC', 'HUBS', 'PANW', 'CDW', 'IPHI', 'ZTS', 'ZEN', 'FSCT', 'FIVN', 'AAPL'],
    '2020-05-31': ['PAYC', 'CDW', 'SSD', 'HUBS', 'ZTS', 'PRO', 'NEWR', 'IPHI', 'PANW', 'OTIS'],
    '2020-06-30': ['PAYC', 'HUBS', 'NEWR', 'CDW', 'PRO', 'SNAP', 'BL', 'ZTS', 'FIVN', 'AAPL'],
    '2020-07-31': ['PAYC', 'HUBS', 'NEWR', 'BAND', 'AAPL', 'FIVN', 'ZTS', 'CDW', 'BL', 'INTU'],
    '2020-08-31': ['PAYC', 'HUBS', 'BAND', 'SNAP', 'AAPL', 'FIVN', 'APPF', 'NEWR', 'BL', 'TXN'],
    '2020-09-30': ['HUBS', 'BAND', 'DDOG', 'BL', 'SNAP', 'APPF', 'AAPL', 'INTU', 'SQ', 'ZTS'],
    '2020-10-31': ['HUBS', 'DDOG', 'SNAP', 'BAND', 'BL', 'INTU', 'SQ', 'APPF', 'AAPL', 'NEWR'],
    '2020-11-30': ['HUBS', 'DDOG', 'NOW', 'APPF', 'BAND', 'SQ', 'NET', 'BL', 'INTU', 'TTC'],
    '2020-12-31': ['PAYC', 'UPWK', 'DDOG', 'TTC', 'AAPL', 'HUBS', 'WDAY', 'VERX', 'PD', 'INTU'],
}

In [81]:
tickers = {
    '2019-01-31': ['FTNT', 'GH', 'ROL', 'ISRG', 'INGN', 'GKOS', 'PRLB', 'PAYC', 'TNDM', 'HQY'],
    '2019-02-28': ['FTNT', 'ROL', 'PAYC', 'HQY', 'LANC', 'PRLB', 'ISRG', 'CGNX', 'GKOS', 'ANET'],
    '2019-03-31': ['ROL', 'BA', 'GH', 'PAYC', 'GKOS', 'ISRG', 'PRLB', 'TNDM', 'BLKB', 'FFIV'],
    '2019-04-30': ['ROL', 'BA', 'ISRG', 'BLKB', 'PAYC', 'GKOS', 'TNDM', 'ABMD', 'INGN', 'MXIM'],
    '2019-05-31': ['ROL', 'BA', 'GKOS', 'BLKB', 'ISRG', 'TNDM', 'PRLB', 'ABMD', 'PAYC', 'XLNX'],
    '2019-06-30': ['PAYC', 'ROL', 'FTNT', 'BA', 'CGNX', 'PRLB', 'GKOS', 'ISRG', 'GH', 'CDW'],
    '2019-07-31': ['PAYC', 'ROL', 'BA', 'ZS', 'DOCU', 'FTNT', 'PRLB', 'AYX', 'MDB', 'CDW'],
    '2019-08-31': ['PAYC', 'AYX', 'DOCU', 'PRLB', 'MDB', 'XLNX', 'CDW', 'SSD', 'HUBS', 'ZS'],
    '2019-09-30': ['PAYC', 'MDB', 'AYX', 'HUBS', 'CDW', 'EVBG', 'NEWR', 'DOCU', 'PANW', 'XLNX'],
    '2019-10-31': ['PAYC', 'HUBS', 'AYX', 'NEWR', 'EVBG', 'SSD', 'PANW', 'DOCU', 'XLNX', 'MDB'],
    '2019-11-30': ['PAYC', 'NEWR', 'HUBS', 'EVBG', 'RMD', 'PANW', 'QTWO', 'DOCU', 'PRO', 'AYX'],
    '2019-12-31': ['CDW', 'NEWR', 'DOCU', 'RMD', 'PANW', 'PAYC', 'HUBS', 'MDB', 'QTWO', 'PRO'],
}

In [84]:
tickers = {
    '2018-01-31': ['CGNX', 'BA', 'PANW', 'FFIV', 'MRVL', 'TTWO', 'HQY', 'AXDX', 'SQ', 'EXAS'],
    '2018-02-28': ['CGNX', 'BA', 'EXAS', 'MRVL', 'PANW', 'HQY', 'FTNT', 'FFIV', 'SQ', 'POWI'],
    '2018-03-31': ['MRVL', 'FFIV', 'HQY', 'BA', 'PANW', 'NEOG', 'PAYC', 'MON_delisted', 'INGN', 'SWKS'],
    '2018-04-30': ['FFIV', 'BA', 'PANW', 'NEOG', 'OKTA', 'PAYC', 'TTC', 'INGN', 'ROL', 'APPF'],
    '2018-05-31': ['ISRG', 'PANW', 'FFIV', 'CGNX', 'SGEN', 'NEOG', 'PAYC', 'OKTA', 'BA', 'ABAX'],
    '2018-06-30': ['FTNT', 'OKTA', 'CGNX', 'EXEL', 'NEOG', 'ISRG', 'PANW', 'BA', 'INGN', 'PAYC'],
    '2018-07-31': ['EXEL', 'ISRG', 'FTNT', 'INGN', 'HQY', 'NEOG', 'PEN', 'MED', 'BA', 'PANW'],
    '2018-08-31': ['FTNT', 'GKOS', 'ISRG', 'INGN', 'HQY', 'NEWR', 'PEN', 'EXEL', 'PANW', 'MED'],
    '2018-09-30': ['FTNT', 'HQY', 'ISRG', 'GKOS', 'CGNX', 'INGN', 'ROL', 'MED', 'PEN', 'ANET'],
    '2018-10-31': ['GKOS', 'ISRG', 'INGN', 'PEN', 'FTNT', 'MED', 'VICR', 'INCY', 'SMAR', 'HQY'],
    '2018-11-30': ['ROL', 'FTNT', 'INGN', 'ISRG', 'GKOS', 'HQY', 'PEN', 'VAR', 'SMAR', 'EPAM'],
    '2018-12-31': ['FTNT', 'GH', 'PAYC', 'ISRG', 'ROL', 'INGN', 'GKOS', 'HQY', 'TNDM', 'PRLB'],
}

In [87]:
tickers = {
    '2017-01-31': ['AA', 'AAL', 'AAPL', 'ABBV', 'ABMD', 'ABT', 'ACHC', 'ACIA', 'ACIW', 'ACN'],
    '2017-02-28': ['AA', 'AAL', 'AAPL', 'ABBV', 'ABMD', 'ABT', 'ACHC', 'ACIA', 'ACIW', 'ACN'],
    '2017-03-31': ['CCMP', 'SSD', 'HP', 'UNF', 'SHO', 'MRVL', 'SLAB', 'TER', 'FNSR', 'GMED'],
    '2017-04-30': ['UNF', 'SSD', 'ICUI', 'INGN', 'MRVL', 'GKOS', 'GMED', 'SHO', 'HP', 'EXAS'],
    '2017-05-31': ['INGN', 'ICUI', 'SSD', 'UNF', 'EXAS', 'HQY', 'CDE', 'PRLB', 'MRVL', 'GMED'],
    '2017-06-30': ['INGN', 'CGNX', 'SGEN', 'IRBT', 'PEN', 'HQY', 'POWI', 'MRVL', 'AMBA', 'ABMD'],
    '2017-07-31': ['CGNX', 'IRBT', 'INGN', 'SGEN', 'MRVL', 'PEN', 'POWI', 'SWKS', 'GMED', 'LOPE'],
    '2017-08-31': ['PEN', 'SGEN', 'INGN', 'ISRG', 'POWI', 'IRBT', 'EXAS', 'MRVL', 'ROL', 'SWKS'],
    '2017-09-30': ['CGNX', 'INGN', 'ACIA', 'ISRG', 'MRVL', 'PEN', 'POWI', 'ROL', 'SWKS', 'PANW'],
    '2017-10-31': ['CGNX', 'PEN', 'INGN', 'MRVL', 'ISRG', 'NEOG', 'POWI', 'SWKS', 'ACIA', 'SGEN'],
    '2017-11-30': ['CGNX', 'ISRG', 'PEN', 'MRVL', 'AXDX', 'POWI', 'GMED', 'SWKS', 'HQY', 'BA'],
    '2017-12-31': ['CGNX', 'ISRG', 'EXAS', 'MRVL', 'BA', 'PANW', 'HQY', 'GRPN', 'POWI', 'IPGP'],
}

In [135]:
tickers = {
    '2024-01-31': ['KLAC', 'SNOW', 'HUBS', 'SNPS', 'AAPL', 'LRCX', 'ADBE', 'FIX', 'BMI', 'FAST'],
    '2024-02-29': ['KLAC', 'SNOW', 'LRCX', 'FAST', 'AAPL', 'FIX', 'BMI', 'MSI', 'MUSA', 'SFM'],
    '2024-03-31': ['SNOW', 'EME', 'KLAC', 'HUBS', 'MANH', 'DY', 'FIX', 'BMI', 'AAPL', 'FAST'],
    '2024-04-30': ['HUBS', 'KLAC', 'MANH', 'EME', 'FAST', 'LRCX', 'AAPL', 'GWW', 'MDB', 'ZS'],
    '2024-05-31': ['DY', 'HUBS', 'KLAC', 'EME', 'MUSA', 'MANH', 'FAST', 'ZS', 'LRCX', 'AAPL'],
    '2024-06-30': ['DY', 'KLAC', 'EME', 'AAPL', 'LBRT', 'HUBS', 'GMS', 'FAST', 'RS', 'MANH'],
    '2024-07-31': ['DY', 'IESC', 'EME', 'KLAC', 'HURN', 'MUSA', 'UI', 'FIX', 'DINO', 'GMS'],
    '2024-08-31': ['DY', 'KLAC', 'EME', 'FIX', 'UI', 'HUBS', 'AAPL', 'MANH', 'LRCX', 'MUSA'],
    '2024-09-30': ['EME', 'DY', 'NTAP', 'KLAC', 'HUBS', 'FIX', 'IESC', 'PWR', 'MANH', 'COR'],
    '2024-10-31': ['KLAC', 'DY', 'FIX', 'EME', 'NTAP', 'MANH', 'IESC', 'AAPL', 'UI', 'LRCX'],
    '2024-11-30': ['FIX', 'EME', 'DY', 'KLAC', 'MUSA', 'PWR', 'NTAP', 'AAPL', 'MANH', 'BCO'],
    '2024-12-31': ['FIX', 'EME', 'DY', 'KLAC', 'GMS', 'BLDR', 'PWR', 'NTAP', 'IESC', 'WM'],
}

In [132]:
tickers = {
    '2023-01-31': ['UNH', 'COR', 'WMT', 'UNP', 'WM', 'KLAC', 'ADP', 'ENSG', 'ABBV', 'JBHT'],
    '2023-02-28': ['UNH', 'WM', 'ABBV', 'WMT', 'WAT', 'COR', 'ENSG', 'DLTR', 'PCTY', 'DE'],
    '2023-03-31': ['UNH', 'WM', 'KLAC', 'WMT', 'CAT', 'FIX', 'ADP', 'MTD', 'DE', 'COR'],
    '2023-04-30': ['WMT', 'UNH', 'KLAC', 'WM', 'FIX', 'MTD', 'KR', 'NSIT', 'PCTY', 'ADP'],
    '2023-05-31': ['MTD', 'KLAC', 'UNH', 'WMT', 'FAST', 'IDXX', 'DE', 'WM', 'AAPL', 'ADP'],
    '2023-06-30': ['XGN', 'KLAC', 'WMT', 'FAST', 'CGNX', 'FIX', 'DE', 'WM', 'PCTY', 'DT'],
    '2023-07-31': ['XGN', 'KLAC', 'FIX', 'FAST', 'WMT', 'GWW', 'TGT', 'SNPS', 'CGNX', 'LRCX'],
    '2023-08-31': ['KLAC', 'DY', 'EME', 'LRCX', 'FAST', 'FELE', 'CBZ', 'UFPI', 'FIX', 'TGT'],
    '2023-09-30': ['KLAC', 'DY', 'FIX', 'EME', 'LRCX', 'FAST', 'CBZ', 'VMI', 'GMS', 'FELE'],
    '2023-10-31': ['DY', 'EME', 'KLAC', 'GMS', 'FIX', 'CBZ', 'VMI', 'ASGN', 'LRCX', 'AAPL'],
    '2023-11-30': ['KLAC', 'BMI', 'EME', 'SNPS', 'GMS', 'LRCX', 'FIX', 'FAST', 'SAIA', 'AAPL'],
    '2023-12-31': ['KLAC', 'EME', 'SNPS', 'AAPL', 'XGN', 'GWW', 'FIX', 'BMI', 'SAIA', 'FAST'],
}

In [112]:
tickers = {
    '2022-01-31': ['PCTY', 'MTD', 'PAYC', 'SPSC', 'NOW', 'SNPS', 'TTD', 'MANH', 'QLYS', 'WAT'],
    '2022-02-28': ['PCTY', 'MTD', 'SPSC', 'PAYC', 'TTD', 'SNPS', 'MANH', 'CDNS', 'NUTX', 'QLYS'],
    '2022-03-31': ['MTD', 'AAPL', 'PCTY', 'WAT', 'SNPS', 'STX', 'IDXX', 'ZTS', 'THC', 'SPSC'],
    '2022-04-30': ['MTD', 'AAPL', 'ESTA', 'PCTY', 'CRS', 'SPR', 'BE', 'IDXX', 'COST', 'THC'],
    '2022-05-31': ['MTD', 'AAPL', 'COST', 'SNPS', 'STX', 'WM', 'SHC', 'THC', 'IDXX', 'WAT'],
    '2022-06-30': ['AAPL', 'MTD', 'WAT', 'WM', 'STX', 'THC', 'IDXX', 'SNPS', 'ACN', 'UNH'],
    '2022-07-31': ['AAPL', 'MTD', 'WAT', 'WM', 'CCMP', 'UNH', 'STX', 'SNPS', 'URI', 'THC'],
    '2022-08-31': ['MTD', 'IDXX', 'UNH', 'WM', 'WAT', 'SNPS', 'AAPL', 'STX', 'CSX', 'UNP'],
    '2022-09-30': ['UNH', 'WM', 'MTD', 'ZTS', 'RSG', 'AAPL', 'CSX', 'UNP', 'IDXX', 'URI'],
    '2022-10-31': ['UNH', 'WM', 'AAPL', 'MTD', 'CVS', 'RSG', 'CSX', 'UNP', 'COR', 'CNC'],
    '2022-11-30': ['UNH', 'WM', 'SUNW', 'ZTS', 'WAT', 'MTD', 'COR', 'CVS', 'KLAC', 'UNP'],
    '2022-12-31': ['UNH', 'WM', 'COR', 'UNP', 'CVS', 'WAT', 'ZTS', 'ABBV', 'ENSG', 'PCTY'],
}

In [119]:
tickers = {
    '2021-01-31': ['PAYC', 'UPWK', 'HUBS', 'DDOG', 'INTU', 'BAND', 'MRNA', 'NEWR', 'PD', 'VERX'],
    '2021-02-28': ['TTD', 'PCTY', 'TYL', 'PAYC', 'APPS', 'MANH', 'MTD', 'CDNS', 'SGEN', 'ZTS'],
    '2021-03-31': ['PCTY', 'TTD', 'PAYC', 'MGNI', 'TYL', 'APPS', 'AMD', 'ESTC', 'MANH', 'CDNS'],
    '2021-04-30': ['PCTY', 'TTD', 'PAYC', 'APPS', 'TYL', 'AMD', 'MANH', 'MGNI', 'SQ', 'NOW'],
    '2021-05-31': ['PCTY', 'TTD', 'PAYC', 'APPS', 'MTD', 'MANH', 'NOW', 'AMD', 'AAPL', 'CDNS'],
    '2021-06-30': ['PCTY', 'TTD', 'MTD', 'APPS', 'PAYC', 'MANH', 'SQ', 'AAPL', 'AMD', 'VEEV'],
    '2021-07-31': ['MTD', 'PCTY', 'SQ', 'AMD', 'MANH', 'TTD', 'AAPL', 'PAYC', 'SWKS', 'APPS'],
    '2021-08-31': ['PCTY', 'MTD', 'SQ', 'AMD', 'TTD', 'APPS', 'MANH', 'PAYC', 'META', 'PINS'],
    '2021-09-30': ['PCTY', 'MTD', 'SPSC', 'PAYC', 'MANH', 'TTD', 'SQ', 'DVA', 'META', 'AMD'],
    '2021-10-31': ['PCTY', 'MTD', 'MANH', 'SPSC', 'PAYC', 'TTD', 'SQ', 'SNPS', 'QLYS', 'VEEV'],
    '2021-11-30': ['PCTY', 'MTD', 'PAYC', 'SPSC', 'TTD', 'SNPS', 'WAT', 'SQ', 'QLYS', 'AMD'],
    '2021-12-31': ['MTD', 'PCTY', 'WAT', 'PAYC', 'SPSC', 'AMD', 'SNPS', 'PGNY', 'IDXX', 'GNRC'],
}

In [123]:
tickers = {
    '2020-01-31': ['PAYC', 'CDW', 'RMD', 'PANW', 'NEWR', 'QTWO', 'HUBS', 'FIVN', 'PRO', 'SNAP'],
    '2020-02-29': ['PAYC', 'CDW', 'PRO', 'PANW', 'HUBS', 'RMD', 'EVBG', 'MDB', 'ZEN', 'FIVN'],
    '2020-03-31': ['PAYC', 'PANW', 'CDW', 'ZTS', 'ADP', 'PAYX', 'FIVN', 'HUBS', 'MDB', 'DOCU'],
    '2020-04-30': ['PAYC', 'HUBS', 'PANW', 'IPHI', 'CDW', 'FSCT', 'SSD', 'APPF', 'FIVN', 'ZEN'],
    '2020-05-31': ['PAYC', 'SSD', 'HUBS', 'PANW', 'IPHI', 'VICR', 'PRO', 'SNAP', 'NEWR', 'XLNX'],
    '2020-06-30': ['PAYC', 'HUBS', 'NEWR', 'SNAP', 'PRO', 'INTU', 'NVDA', 'FIVN', 'APPF', 'BAND'],
    '2020-07-31': ['PAYC', 'HUBS', 'NEWR', 'BAND', 'SNAP', 'FIVN', 'INTU', 'APPF', 'ZEN', 'BL'],
    '2020-08-31': ['PAYC', 'HUBS', 'FIVN', 'SNAP', 'BAND', 'NEWR', 'NET', 'VICR', 'INTU', 'APPF'],
    '2020-09-30': ['HUBS', 'BAND', 'DDOG', 'SNAP', 'SQ', 'BL', 'INTU', 'APPF', 'PCTY', 'MNTV'],
    '2020-10-31': ['HUBS', 'DDOG', 'SNAP', 'BAND', 'VICR', 'INTU', 'SQ', 'NEWR', 'BL', 'APPF'],
    '2020-11-30': ['HUBS', 'DDOG', 'NET', 'NOW', 'BAND', 'VICR', 'APPF', 'UPWK', 'SQ', 'ADP'],
    '2020-12-31': ['PAYC', 'UPWK', 'MRNA', 'TTC', 'HUBS', 'INTU', 'DDOG', 'AAPL', 'NOW', 'PD'],
}

In [126]:
tickers = {
    '2019-01-31': ['FTNT', 'ROL', 'GH', 'INGN', 'GKOS', 'ISRG', 'PRLB', 'PAYC', 'TNDM', 'HQY'],
    '2019-02-28': ['FTNT', 'ROL', 'LANC', 'PAYC', 'PRLB', 'HQY', 'CGNX', 'ISRG', 'GKOS', 'ANET'],
    '2019-03-31': ['ROL', 'BA', 'PAYC', 'GKOS', 'GH', 'ISRG', 'PRLB', 'TNDM', 'BLKB', 'INGN'],
    '2019-04-30': ['ROL', 'PAYC', 'BA', 'BLKB', 'ISRG', 'GKOS', 'TNDM', 'ABMD', 'INGN', 'ENR'],
    '2019-05-31': ['BA', 'BLKB', 'ROL', 'PAYC', 'GKOS', 'ENR', 'ISRG', 'AAPL', 'PS', 'MSFT'],
    '2019-06-30': ['PAYC', 'ROL', 'BA', 'FTNT', 'CDW', 'BLKB', 'AYX', 'QLYS', 'CGNX', 'RPD'],
    '2019-07-31': ['PAYC', 'BA', 'ROL', 'DOCU', 'CDW', 'NEWR', 'MDB', 'ZS', 'AYX', 'RHT'],
    '2019-08-31': ['PAYC', 'DOCU', 'AYX', 'MDB', 'CDW', 'PANW', 'XLNX', 'HUBS', 'NEWR', 'EVBG'],
    '2019-09-30': ['PAYC', 'NEWR', 'MDB', 'EVBG', 'HUBS', 'AYX', 'XLNX', 'UNH', 'CDW', 'DOCU'],
    '2019-10-31': ['PAYC', 'DOCU', 'NEWR', 'PANW', 'HUBS', 'AYX', 'EVBG', 'MDB', 'TMO', 'XLNX'],
    '2019-11-30': ['PAYC', 'NEWR', 'HUBS', 'PANW', 'AYX', 'QTWO', 'EVBG', 'DOCU', 'MDB', 'PRO'],
    '2019-12-31': ['RMD', 'PAYC', 'DOCU', 'HUBS', 'PANW', 'CDW', 'MDB', 'NEWR', 'ZTS', 'EVBG'],
}

In [129]:
tickers = {
    '2018-01-31': ['CGNX', 'BA', 'PANW', 'MRVL', 'SQ', 'HQY', 'FFIV', 'FTNT', 'EXAS', 'AXDX'],
    '2018-02-28': ['CGNX', 'BA', 'EXAS', 'FTNT', 'PANW', 'MRVL', 'FFIV', 'HQY', 'TTWO', 'GRPN'],
    '2018-03-31': ['MRVL', 'FFIV', 'BA', 'HQY', 'NEOG', 'PANW', 'MON_delisted', 'PAYC', 'INGN', 'TTC'],
    '2018-04-30': ['BA', 'FFIV', 'PANW', 'NEOG', 'PAYC', 'INGN', 'OKTA', 'APPF', 'SWKS', 'TTC'],
    '2018-05-31': ['ISRG', 'OKTA', 'CGNX', 'PANW', 'FFIV', 'SGEN', 'BA', 'PAYC', 'NEOG', 'ABAX'],
    '2018-06-30': ['FTNT', 'CGNX', 'ISRG', 'OKTA', 'EXEL', 'NEOG', 'PANW', 'INGN', 'BA', 'CORT'],
    '2018-07-31': ['EXEL', 'ISRG', 'FTNT', 'INGN', 'HQY', 'BA', 'PEN', 'NEOG', 'ABMD', 'PANW'],
    '2018-08-31': ['FTNT', 'HQY', 'GKOS', 'ISRG', 'NEWR', 'INGN', 'PANW', 'PEN', 'EXEL', 'CGNX'],
    '2018-09-30': ['FTNT', 'HQY', 'ISRG', 'ANET', 'GKOS', 'INGN', 'ROL', 'CGNX', 'PEN', 'MED'],
    '2018-10-31': ['GKOS', 'ISRG', 'INGN', 'MED', 'FTNT', 'PEN', 'VICR', 'SMAR', 'HQY', 'INCY'],
    '2018-11-30': ['FTNT', 'ROL', 'INGN', 'ISRG', 'GKOS', 'HQY', 'PS', 'PEN', 'PRLB', 'PAYC'],
    '2018-12-31': ['FTNT', 'GH', 'INGN', 'GKOS', 'ROL', 'PAYC', 'ISRG', 'TNDM', 'HQY', 'PRLB'],
}

In [136]:
import pandas as pd
import yfinance as yf
from datetime import timedelta

# --- 1. HELPER: GET MARKET RETURN (S&P 500) ---
def get_market_return(buy_date, sell_date, ticker="^GSPC"):
    try:
        market = yf.Ticker(ticker)
        start_dt = pd.to_datetime(buy_date)
        end_dt = pd.to_datetime(sell_date)
        
        # Fetch history
        hist = market.history(start=start_dt, end=end_dt + timedelta(days=5), auto_adjust=False)
        
        # *** FIX: REMOVE TIMEZONE ***
        hist.index = hist.index.tz_localize(None)
        
        if hist.empty:
            return 0.0
            
        # Get Price Col
        col = 'Adj Close' if 'Adj Close' in hist.columns else 'Close'
        
        # Find closest row to buy date (Index is now naive, so this comparison works)
        mask_buy = hist.index >= start_dt
        if not mask_buy.any(): return 0.0
        buy_row = hist.loc[mask_buy].iloc[0]
        
        # Find closest row to sell date
        mask_sell = hist.index <= (end_dt + timedelta(days=1))
        if not mask_sell.any(): return 0.0
        sell_row = hist.loc[mask_sell].iloc[-1]
        
        return ((sell_row[col] - buy_row[col]) / buy_row[col]) * 100
    except Exception as e:
        print(f"Error fetching market data: {e}")
        return 0.0

# --- 2. MAIN PORTFOLIO FUNCTION ---
def get_portfolio_performance_yahoo(tickers, buy_date, sell_date, initial_investment):
    investment_per_stock = initial_investment / len(tickers)
    results = []
    
    start_dt = pd.to_datetime(buy_date)
    end_dt = pd.to_datetime(sell_date)

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_dt, end=end_dt + timedelta(days=5), auto_adjust=False)
            
            # *** FIX: REMOVE TIMEZONE ***
            hist.index = hist.index.tz_localize(None)

            if hist.empty:
                print(f"  [Warn] {ticker}: No data")
                continue

            col = 'Adj Close' if 'Adj Close' in hist.columns else 'Close'

            # Get Buy Price (First available after start date)
            mask_buy = hist.index >= start_dt
            if not mask_buy.any(): continue
            buy_price = hist.loc[mask_buy].iloc[0][col]

            # Get Sell Price (Last available before/on sell date)
            mask_sell = hist.index <= (end_dt + timedelta(days=1))
            if not mask_sell.any(): continue
            sell_price = hist.loc[mask_sell].iloc[-1][col]

            # Calculate Return
            pct_return = ((sell_price - buy_price) / buy_price) * 100
            profit = investment_per_stock * (pct_return / 100)

            results.append({
                'Ticker': ticker,
                'Buy Price': round(buy_price, 2), 
                'Sell Price': round(sell_price, 2),
                'Return (%)': round(pct_return, 2),
                'P/L ($)': round(profit, 2)
            })

        except Exception as e:
            # print(f"  [Error] {ticker}: {e}") # Uncomment to debug individual stocks
            continue

    # Summary
    df = pd.DataFrame(results)
    if df.empty: return df, 0.0, 0.0
    
    total_pl = df['P/L ($)'].sum()
    total_return = (total_pl / initial_investment) * 100
    return df, total_pl, total_return

# --- 2. MAIN PORTFOLIO FUNCTION ---
def get_portfolio_performance(tickers, buy_date, sell_date, initial_investment, source='simfin', simfin_data=None):
    investment_per_stock = initial_investment / len(tickers)
    results = []
    
    start_dt = pd.to_datetime(buy_date)
    end_dt = pd.to_datetime(sell_date)

    # ------------------------------------------
    # PRE-LOAD SIMFIN DATA (If selected)
    # ------------------------------------------
    df_simfin = None
    if source == 'simfin':
        # Default path if not provided
        if simfin_data is None:
            simfin_data = '/Volumes/Simfin/my_cache/us-shareprices-daily.csv'
        
        # Load data if a path is provided
        if isinstance(simfin_data, str):
            df_simfin = pd.read_csv(
                simfin_data, 
                sep=';', 
                usecols=['Ticker', 'Date', 'Adj. Close'], 
                parse_dates=['Date']
            )
        else:
            df_simfin = simfin_data

    for ticker in tickers:
        try:
            buy_price = 0.0
            sell_price = 0.0

            # ------------------------------------------
            # OPTION A: SIMFIN DATABASE
            # ------------------------------------------
            if source == 'simfin':
                # Filter for ticker
                mask_ticker = (df_simfin['Ticker'] == ticker)
                if not mask_ticker.any():
                    print(f"  [Warn] {ticker}: No SimFin data")
                    continue
                
                df_subset = df_simfin[mask_ticker].sort_values('Date')
                
                # Buy Price
                mask_buy = df_subset['Date'] >= start_dt
                if not mask_buy.any(): continue
                buy_price = df_subset.loc[mask_buy].iloc[0]['Adj. Close']
                
                # Sell Price
                mask_sell = df_subset['Date'] <= end_dt
                if not mask_sell.any(): continue
                sell_price = df_subset.loc[mask_sell].iloc[-1]['Adj. Close']

            # ------------------------------------------
            # OPTION B: YAHOO FINANCE (Original Logic)
            # ------------------------------------------
            else:
                stock = yf.Ticker(ticker)
                hist = stock.history(start=start_dt, end=end_dt + timedelta(days=5), auto_adjust=False)
                
                # *** FIX: REMOVE TIMEZONE ***
                hist.index = hist.index.tz_localize(None)

                if hist.empty:
                    print(f"  [Warn] {ticker}: No data")
                    continue

                col = 'Adj Close' if 'Adj Close' in hist.columns else 'Close'

                # Get Buy Price (First available after start date)
                mask_buy = hist.index >= start_dt
                if not mask_buy.any(): continue
                buy_price = hist.loc[mask_buy].iloc[0][col]

                # Get Sell Price (Last available before/on sell date)
                mask_sell = hist.index <= (end_dt + timedelta(days=1))
                if not mask_sell.any(): continue
                sell_price = hist.loc[mask_sell].iloc[-1][col]

            # ------------------------------------------
            # CALCULATE RETURNS (Shared)
            # ------------------------------------------
            pct_return = ((sell_price - buy_price) / buy_price) * 100
            profit = investment_per_stock * (pct_return / 100)

            results.append({
                'Ticker': ticker,
                'Buy Price': round(buy_price, 2), 
                'Sell Price': round(sell_price, 2),
                'Return (%)': round(pct_return, 2),
                'P/L ($)': round(profit, 2)
            })

        except Exception as e:
            # print(f"  [Error] {ticker}: {e}") # Uncomment to debug individual stocks
            continue

    # Summary
    df = pd.DataFrame(results)
    if df.empty: return df, 0.0, 0.0
    
    total_pl = df['P/L ($)'].sum()
    total_return = (total_pl / initial_investment) * 100
    return df, total_pl, total_return

# --- 3. EXECUTION LOOP ---

print(f"{'='*70}")
print(f"STARTING MONTHLY BACKTEST vs S&P 500 (1-Year Holding)")
print(f"{'='*70}\n")

backtest_summary = {}

# Assuming 'tickers' dictionary is defined from previous steps
# If starting fresh, uncomment/edit this dummy dict:
# tickers = {'2024-01-31': ['AAPL', 'MSFT', 'GOOG'], '2024-02-29': ['NVDA', 'AMD']} 

for buy_date in sorted(tickers.keys()):
    ticker_list = tickers[buy_date]
    if not ticker_list: continue

    # Calculate Sell Date
    buy_dt_obj = pd.to_datetime(buy_date)
    sell_dt_obj = buy_dt_obj + pd.DateOffset(years=1)
    sell_date = sell_dt_obj.strftime('%Y-%m-%d')
    
    # Run Backtests
    performance_df, total_dollar_pl, total_pct = get_portfolio_performance(
        tickers=ticker_list, 
        buy_date=buy_date, 
        sell_date=sell_date, 
        initial_investment=10000
    )
    
    sp500_pct = get_market_return(buy_date, sell_date, ticker="^GSPC")
    alpha = total_pct - sp500_pct
    
    # Print Batch
    print(f"\n>>> Period: {buy_date} -> {sell_date}")
    print(f"Portfolio Return:  {total_pct:>6.2f}%")
    print(f"S&P 500 Return:    {sp500_pct:>6.2f}%")
    print(f"Alpha (Edge):      {alpha:>6.2f}%  {'✅' if alpha > 0 else '❌'}")
    print("-" * 40)

    # Store
    backtest_summary[buy_date] = {
        'buy_date': buy_date,
        'sell_date': sell_date,
        'portfolio_return_pct': round(total_pct, 2),
        'sp500_return_pct': round(sp500_pct, 2),
        'alpha': round(alpha, 2),
        'stocks': ticker_list
    }

# --- FINAL STATS ---
print("\n" + "="*70)
print("FINAL SUMMARY STATISTICS")
print("="*70)

if backtest_summary:
    wins = sum(1 for d in backtest_summary.values() if d['alpha'] > 0)
    avg_port = sum(d['portfolio_return_pct'] for d in backtest_summary.values()) / len(backtest_summary)
    avg_sp500 = sum(d['sp500_return_pct'] for d in backtest_summary.values()) / len(backtest_summary)
    
    print(f"Total Periods:   {len(backtest_summary)}")
    print(f"Win Rate:        {wins}/{len(backtest_summary)} ({wins/len(backtest_summary)*100:.1f}%)")
    print(f"Avg Port Return: {avg_port:.2f}%")
    print(f"Avg S&P Return:  {avg_sp500:.2f}%")
    print(f"Avg Alpha:       {avg_port - avg_sp500:.2f}%")



STARTING MONTHLY BACKTEST vs S&P 500 (1-Year Holding)


>>> Period: 2024-01-31 -> 2025-01-31
Portfolio Return:   20.36%
S&P 500 Return:     24.66%
Alpha (Edge):       -4.30%  ❌
----------------------------------------

>>> Period: 2024-02-29 -> 2025-02-28
Portfolio Return:   25.98%
S&P 500 Return:     16.84%
Alpha (Edge):        9.14%  ✅
----------------------------------------

>>> Period: 2024-03-31 -> 2025-03-31
Portfolio Return:    1.66%
S&P 500 Return:      7.42%
Alpha (Edge):       -5.77%  ❌
----------------------------------------

>>> Period: 2024-04-30 -> 2025-04-30
Portfolio Return:    2.06%
S&P 500 Return:     11.29%
Alpha (Edge):       -9.23%  ❌
----------------------------------------

>>> Period: 2024-05-31 -> 2025-05-31
Portfolio Return:   11.31%
S&P 500 Return:     12.02%
Alpha (Edge):       -0.71%  ❌
----------------------------------------

>>> Period: 2024-06-30 -> 2025-06-30
Portfolio Return:   12.20%
S&P 500 Return:     13.20%
Alpha (Edge):       -1.00%  ❌
--------

In [137]:
# Convert the dictionary to a DataFrame
results_df = pd.DataFrame.from_dict(backtest_summary, orient='index')

# Select and Rename Columns for Excel
excel_output = results_df[[
    'buy_date', 
    'sell_date', 
    'portfolio_return_pct', 
    'sp500_return_pct', 
    'alpha',
    'stocks'
]].copy()

excel_output.columns = [
    'Start Date', 
    'End Date', 
    'Portfolio Return (%)', 
    'S&P 500 Return (%)', 
    'Alpha (%)', 
    'Stocks Picked'
]

# Calculate Averages row
avg_row = pd.DataFrame({
    'Start Date': ['AVERAGE'], 
    'End Date': ['-'],
    'Portfolio Return (%)': [excel_output['Portfolio Return (%)'].mean()],
    'S&P 500 Return (%)': [excel_output['S&P 500 Return (%)'].mean()],
    'Alpha (%)': [excel_output['Alpha (%)'].mean()],
    'Stocks Picked': ['-']
})

# Combine and Export
final_df = pd.concat([excel_output, avg_row], ignore_index=True)

# 5. DYNAMIC FILENAME (BASED ON BUY DATE ONLY)
# We look at the first buy date and the last buy date.
start_year = str(results_df['buy_date'].min())[:4]
end_year = str(results_df['buy_date'].max())[:4]
    
if start_year == end_year:
    # Example: Backtest_2024.xlsx
    filename = f"Backtest_{start_year}.xlsx"
else:
    # Example: Backtest_2017-2024.xlsx
    filename = f"Backtest_{start_year}-{end_year}.xlsx"

# 6. Save
final_df.to_excel(filename, index=False)
print(f"\n[Success] Results saved to: {filename}")



[Success] Results saved to: Backtest_2024.xlsx
